In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
#Advanced Fork-Join Example 13
#Complex fork-join with 3 parallel branches
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)
queue3 = Queue(model,'Queue3',SchedStrategy.PS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)
jobclass2 = ClosedClass(model, 'class2', 10, delay, 0)

In [4]:
# Service configurations
# Class 1
queue1.set_service(jobclass1, Exp(1.5))
queue2.set_service(jobclass1, Exp(1.1))
queue3.set_service(jobclass1, Exp(2.5))
delay.set_service(jobclass1, Exp(0.5))

# Class 2
queue1.set_service(jobclass2, Exp(2.8))
queue2.set_service(jobclass2, Exp(3.0))
queue3.set_service(jobclass2, Exp(1.0))
delay.set_service(jobclass2, Exp(0.8))

In [5]:
# Set routing matrix with serial routing queue2->queue3
P = model.init_routing_matrix()

# Class 1 routing
P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue2, queue3, 1.0)  # Serial: queue2 -> queue3
P.set(jobclass1, jobclass1, queue3, join, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

# Class 2 routing
P.set(jobclass2, jobclass2, delay, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue2, queue3, 1.0)  # Serial: queue2 -> queue3
P.set(jobclass2, jobclass2, queue3, join, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with MVA only (like MATLAB)
solver = MVA(model)
avgTable = solver.avg_table()

Station JobClass    QLen   Util  RespT  ResidT   ArvR   Tput
 Delay1   class1  1.9464 1.9464 2.0000  2.0000 0.9732 0.9732
 Delay1   class2  1.1230 1.1230 1.2500  1.2500 0.8984 0.8984
 Queue1   class1  1.8473 0.6488 1.8982  1.8982 0.9732 0.9732
 Queue1   class2  0.4724 0.3208 0.5259  0.5259 0.8984 0.8984
 Queue2   class1  7.6739 0.8847 7.8854  7.8854 0.9732 0.9732
 Queue2   class2  0.4275 0.2995 0.4758  0.4758 0.8984 0.8984
 Queue3   class1  0.6374 0.3893 0.6550  0.6550 0.9732 0.9732
 Queue3   class2  8.8389 0.8984 9.8389  9.8389 0.8984 0.8984
   Join   class1 14.0023 0.0000 4.7961  4.7961 0.9732 0.9732
   Join   class2 16.8923 0.0000 6.2678  6.2678 0.8984 0.8984
